## **Ingest pitstop.json file**
**Step 1 - Read josn file\
Step 2 - Rename and add the columns\
Step 3 - Write data into datalake as parquet file**

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

**Step 1 - Read josn file**

In [0]:
from pyspark.sql.types import IntegerType, StringType, StructField, StructType

In [0]:
pit_stops_schema = StructType(fields = [StructField("raceId", IntegerType(), False),
                                      StructField("driverId", IntegerType(), False),
                                      StructField("stop", StringType(), True),
                                      StructField("lap", IntegerType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("duration", StringType(), True),
                                      StructField("milliseconds", IntegerType(), True)                                       
])

In [0]:
pit_stops_df = spark.read\
    .schema(pit_stops_schema)\
        .option("multiLine", True)\
            .json(f"{raw_folder_path}/{v_file_date}/pit_stops.json")

**Step 2 - Rename and add the columns**

In [0]:
from pyspark.sql.functions import lit

In [0]:
pit_stops_renamed_df = pit_stops_df.withColumnRenamed("raceId", "race_id")\
    .withColumnRenamed("driverId", "driver_id")\
        .withColumn("data-source", lit(v_data_source))

In [0]:
pit_stops_with_ingestion_date_df = add_ingestion_date(pit_stops_renamed_df)

**Step 3 - Write data into datalake as parquet file**

In [0]:
#pit_stops_with_ingestion_date_df.write\
#  .mode("overwrite")\
#  .parquet(f"{processed_folder_path}/pit_stops")

#pit_stops_with_ingestion_date_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.pit_stops")

In [0]:
#incremental_load(pit_stops_with_ingestion_date_df, 'race_id', 'f1_processed', 'pit_stops')

In [0]:
merge_condition = "tgt.driver_id = src.driver_id and tgt.stop = src.stop and tgt.race_id = src.race_id"

merge_delta_table(pit_stops_with_ingestion_date_df, 'f1_processed', 'pit_stops', processed_folder_path, merge_condition, 'race_id')

In [0]:
%sql
SELECT race_id, COUNT(race_id)
FROM f1_processed.pit_stops
GROUP BY race_id
ORDER BY race_id DESC;

race_id,count(race_id)
1053,56
1052,40
1047,23
1046,39
1045,57
1044,38
1043,30
1042,25
1041,33
1040,24


In [0]:
dbutils.notebook.exit("Success")